<h1 align="center"> Generative AI Hackathon</h1>
<table align="center">
    <td>
        <a href="https://colab.research.google.com/github/teamdatatonic/gen-ai-hackathon/blob/feature%2FDBA-hackathon/analytics_hackathon.ipynb">
            <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo">
            <span style="vertical-align: middle;">Run in Colab</span>
        </a>
    </td>
    <!-- <td>
        <a href="https://github.com/teamdatatonic/gen-ai-hackathon/blob/DBA-hackathon/analytics_hackathon.ipynb">
            <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
            <span style="vertical-align: middle;">View on GitHub</span>
        </a>
    </td> -->
    <!-- <td>
        <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/teamdatatonic/gen-ai-hackathon/main/analytics_hackathon.ipynb">
            <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"> 
            <span style="vertical-align: middle;">Open in Vertex AI Workbench</span>
        </a>
    </td> -->
</table>
<hr>

**➡️ Your task:** Learn about Generative AI by building your own Database Analytics using Python and LangChain!

**❗ Note:** This workshop has been designed to be run in Google CoLab. Support for running the workshop locally or using VertexAI Workbench is provided, but we heavily recommend CoLab for the best experience.

**❗ Note:** If your kernel doesn't restart automatically, click the "Restart Runtime" button above your notebook.
If you dont see a restart button, go to the "Runtime" toolbar tab then "Restart Runtime". After restarting, continue executing the project from below this cell.


## Pip install package dependencies

In [14]:
!poetry install 
!poetry export --format requirements.txt --output requirements.txt
%pip install -r requirements.txt

Installing dependencies from lock file

No dependencies to install or update
Ignoring colorama: markers 'python_version >= "3.9" and python_version < "3.12" and sys_platform == "win32"' don't match your environment
Ignoring greenlet: markers 'python_version >= "3.9" and (platform_machine == "win32" or platform_machine == "WIN32" or platform_machine == "AMD64" or platform_machine == "amd64" or platform_machine == "x86_64" or platform_machine == "ppc64le" or platform_machine == "aarch64") and python_version < "3.12"' don't match your environment

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


**❗ Note:** This notebook will keep running until it is shut down manually.

In [15]:
!poetry run jupyter notebook

[I 2023-09-14 11:41:18.923 ServerApp] Package notebook took 0.0000s to import
[I 2023-09-14 11:41:18.930 ServerApp] Package jupyter_lsp took 0.0074s to import
[W 2023-09-14 11:41:18.930 ServerApp] A `_jupyter_server_extension_points` function was not found in jupyter_lsp. Instead, a `_jupyter_server_extension_paths` function was found and will be used for now. This function name will be deprecated in future releases of Jupyter Server.
[I 2023-09-14 11:41:18.933 ServerApp] Package jupyter_server_terminals took 0.0032s to import
[I 2023-09-14 11:41:18.934 ServerApp] Package jupyterlab took 0.0000s to import
[I 2023-09-14 11:41:18.950 ServerApp] Package notebook_shim took 0.0000s to import
[W 2023-09-14 11:41:18.950 ServerApp] A `_jupyter_server_extension_points` function was not found in notebook_shim. Instead, a `_jupyter_server_extension_paths` function was found and will be used for now. This function name will be deprecated in future releases of Jupyter Server.
[I 2023-09-14 11:41:18

## Accessing the Vertex AI Endpoint

Currently, Vertex AI LLMs are accessible via Google Cloud projects. 

1. Set the env variables `project_id` and `dataset_id` with the filepath (**❗ Note:** the `/content/` folder is where uploaded files are stored by default).

In [ ]:
# Replace 'your-project-id' with your Google Cloud project ID
project_id = 'dt-gen-ai-hackathon-dev'
dataset_id = 'database_analytics_demo_v2'

In [ ]:
import os

# @title Set project credentials. { run: "auto", display-mode: "form" }
# @markdown Set the filepath to the `.json` credentials file.
GOOGLE_APPLICATION_CREDENTIALS = "credentials.json"  # @param {type:"string"}
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = GOOGLE_APPLICATION_CREDENTIALS

In [ ]:
!gcloud config set account dt-gen-ai-hackathon-sa@dt-gen-ai-hackathon-dev.iam.gserviceaccount.com

Updated property [core/account].


In [ ]:
!gcloud auth activate-service-account --key-file={GOOGLE_APPLICATION_CREDENTIALS}

Activated service account credentials for: [dt-gen-ai-hackathon-sa@dt-gen-ai-hackathon-dev.iam.gserviceaccount.com]


In [ ]:
!gcloud config set project {project_id}

Updated property [core/project].


## Setting up the BigQuery client

In [ ]:
from google.cloud import bigquery, aiplatform

# Initialize the BigQuery client
client = bigquery.Client(project=project_id)

# Initialise AI Platform
aiplatform.init(project=project_id)

## Create SQL chain

In [ ]:
from langchain_experimental.sql import SQLDatabaseChain
from langchain.sql_database import SQLDatabase
from langchain.llms import VertexAI
from sqlalchemy.engine import create_engine



engine = create_engine(f"bigquery://{project_id}/{dataset_id}")

llm = VertexAI(model_name='text-bison@001',
               temperature=0, max_output_tokens=1024)

db = SQLDatabase(engine=engine)


db_chain = SQLDatabaseChain.from_llm(
        llm,
        db,
        verbose=True
    )

# Some test questions
# db_chain("How many employees are there?")
# db_chain("total_revenue")
db_chain("How many customers are there?")


cloudpickle is not installed. Please call `pip install google-cloud-aiplatform[preview]`.


> Entering new SQLDatabaseChain chain...
How many customers are there?
SQLQuery:SELECT COUNT(*) FROM customers
SQLResult: [(1000,)]
Answer:There are 1000 customers.
> Finished chain.


{'query': 'How many customers are there?',
 'result': 'There are 1000 customers.'}